### PT – Observação do DataFrame e Aplicação de SMOTE

Nesta etapa, exploramos a base de treino para verificar a distribuição das classes de fraude e regularidade.  
Foi criada uma coluna de rótulo única (`fraude_tipo`) combinando os tipos de fraude: Phantom Billing, Upcoding, Duplicidade de Pagamento e registros regulares.  

Após a verificação, aplicamos **SMOTE** para balancear a base de treino, garantindo **25% de cada classe**, evitando problemas de desbalanceamento e melhorando a performance do modelo de Machine Learning.  

---

### EN – DataFrame Inspection and SMOTE Application

In this step, we explore the training dataset to check the distribution of fraud and regular classes.  
A single label column (`fraude_tipo`) was created, combining the fraud types: Phantom Billing, Upcoding, Duplicate Billing, and regular records.  

After inspection, **SMOTE** was applied to balance the training set, ensuring **25% for each class**, avoiding class imbalance issues and improving the performance of the Machine Learning model.


In [1]:
import pandas as pd
import plotly.express as px

# 1. Carregar o CSV
df = pd.read_csv('treino_dados_sintetico.csv')
df.head(10)

,id_atendimento,beneficiario_id,prestador_id,data_solicitacao,CID_,status_saude,tipo_atendimento,procedimento_codigo,autorizacao_status,data_realizacao,valor_pago,procedimentos_urgencia,phantom_billing,upcoding,duplicidade_pagamento,regular
0,9070,2109,652,2023-07-28,I10,-1,exame,71821,aprovado,2023-08-06,997.29,nao,False,False,False,1
1,2604,5020,361,2024-11-18,C34,-1,consulta,44588,aprovado,0,472.11,nao,True,False,False,0
2,7739,2232,675,2023-07-29,H10,0,exame,43719,pendente,2023-04-27,611.00,nao,False,False,False,1
3,1580,4568,231,2023-07-21,H10,-1,cirurgia,81804,pendente,2024-05-21,799.38,sim,False,False,False,1
4,5059,1913,440,2023-06-20,B20,1,consulta,86624,negado,2024-10-19,832.95,nao,False,False,False,1
5,5754,5951,526,2024-03-08,I10,-1,consulta,47888,aprovado,2023-10-07,391.90,nao,False,False,False,1
6,3549,9328,797,2023-09-04,D50,0,consulta,62105,aprovado,2023-03-02,818.43,nao,False,False,False,1
7,410,3427,997,2023-06-05,B20,-1,consulta,75391,aprovado,2023-07-10,460.53,nao,False,False,False,1
8,4098,8299,985,2024-08-13,E11,0,exame,76839,aprovado,2023-04-10,505.25,nao,False,False,False,1
9,6462,6401,382,2023-05-30,H10,-1,consulta,15571,aprovado,2024-06-11,368.76,nao,False,False,False,1


In [2]:
import pandas as pd
import plotly.express as px

# 1. Carregar o CSV
df = pd.read_csv('treino_dados_sintetico.csv')

# 2. Definir colunas de interesse
colunas = ['phantom_billing', 'upcoding', 'duplicidade_pagamento', 'regular']

# 3. Contar os valores True (ou 1)
contagens = {
    'phantom_billing': df['phantom_billing'].sum(),
    'upcoding': df['upcoding'].sum(),
    'duplicidade_pagamento': df['duplicidade_pagamento'].sum(),
    'regular': df['regular'].sum()
}

# 4. Transformar em DataFrame
contagens_df = pd.DataFrame(list(contagens.items()), columns=['Tipo', 'Quantidade'])

# 5. Criar gráfico de barras
fig = px.bar(contagens_df,
             x='Tipo',
             y='Quantidade',
             title='Distribuição de Tipos de Fraude e Casos Regulares',
             text='Quantidade',
             labels={'Tipo': 'Categoria', 'Quantidade': 'Contagem'})

fig.show()

# SMOTE:

Como será feito:

✅ Criação de coluna de rótulo única (ex: fraude_tipo) para os seguintes valores:

'phantom_billing'

'upcoding'

'duplicidade_pagamento'

'regular'

✅ Filtragem dos dados para que cada linha tenha apenas um tipo de fraude ou seja regular (sem sobreposição entre tipos de fraude).

✅ Aplicação do LabelEncoder na coluna fraude_tipo.

✅ Separação das features (X) e target (y).

✅ Aplicação do SMOTE para criar um dataset balanceado 25% para cada classe.

# SMOTE:

How it will be done:

✅ Create a single label column (e.g., `fraude_tipo`) with the following values:

- 'phantom_billing'  
- 'upcoding'  
- 'duplicidade_pagamento'  
- 'regular'  

✅ Filter the data so that each row has only one type of fraud or is regular (no overlap between fraud types).  

✅ Apply `LabelEncoder` to the `fraude_tipo` column.  

✅ Separate features (X) and target (y).  

✅ Apply SMOTE to create a balanced dataset with 25% for each class.



In [8]:
import pandas as pd

# Carregar o dataset
df = pd.read_csv('treino_dados_sintetico.csv')

# Coluna 'fraude_tipo' garantindo exclusividade para cada fraude:
def classificar_fraude(row):
    if row['phantom_billing']:
        return 'phantom_billing'
    elif row['upcoding']:
        return 'upcoding'
    elif row['duplicidade_pagamento']:
        return 'duplicidade_pagamento'
    else:
        return 'regular'

df['fraude_tipo'] = df.apply(classificar_fraude, axis=1)

In [9]:
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# 1. Codificar variáveis categóricas em X
X = df.drop(['fraude_tipo', 'phantom_billing', 'upcoding', 'duplicidade_pagamento', 'regular'], axis=1)

# Identificar colunas categóricas
colunas_categoricas = X.select_dtypes(include='object').columns

# Aplicar LabelEncoder para cada coluna categórica:
for col in colunas_categoricas:
    le_col = LabelEncoder()
    X[col] = le_col.fit_transform(X[col])

# 2. Codificar y (fraude_tipo)
le_target = LabelEncoder()
y = le_target.fit_transform(df['fraude_tipo'])

# 3. Aplicar SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 4. Recuperar os rótulos originais
y_resampled_labels = le_target.inverse_transform(y_resampled)

# 5. DataFrame final balanceado
df_balanceado = pd.DataFrame(X_resampled, columns=X.columns)
df_balanceado['fraude_tipo'] = y_resampled_labels

In [5]:
print(df_balanceado['fraude_tipo'].value_counts(normalize=True))

fraude_tipo
regular                  0.25
phantom_billing          0.25
duplicidade_pagamento    0.25
upcoding                 0.25
Name: proportion, dtype: float64


In [6]:
import plotly.express as px

# Contar proporção por tipo de fraude
proporcoes = df_balanceado['fraude_tipo'].value_counts(normalize=True)

# Converter para DataFrame
grafico_df = proporcoes.reset_index()
grafico_df.columns = ['Tipo de Fraude', 'Proporcao']

# Criar gráfico de barras
fig = px.bar(grafico_df,
             x='Tipo de Fraude',
             y='Proporcao',
             title='Distribuição Balanceada das Fraudes e Casos Regulares',
             text=grafico_df['Proporcao'].apply(lambda x: f'{x:.2%}'),
             labels={'Proporcao': 'Proporção'})

fig.show()

In [7]:
import plotly.express as px

# Contar valores absolutos e calcular proporção
contagem = df_balanceado['fraude_tipo'].value_counts()
proporcao = df_balanceado['fraude_tipo'].value_counts(normalize=True)

# Criar DataFrame combinado
grafico_df = pd.DataFrame({
    'Tipo de Fraude': contagem.index,
    'Quantidade': contagem.values,
    'Proporcao': proporcao.values
})

# Criar gráfico
fig = px.bar(grafico_df,
             x='Tipo de Fraude',
             y='Quantidade',
             text=grafico_df.apply(lambda row: f"{row['Quantidade']} ({row['Proporcao']:.0%})", axis=1),
             title='Distribuição Balanceada das Fraudes e Casos Regulares',
             labels={'Quantidade': 'Número de Registros'})

fig.update_traces(textposition='outside')

fig.show()

In [10]:
df_balanceado.to_csv('dados_balanceados_SMOTE.csv', index=False)

In [11]:
df_balanceado.head(10)

,id_atendimento,beneficiario_id,prestador_id,data_solicitacao,CID_,status_saude,tipo_atendimento,procedimento_codigo,autorizacao_status,data_realizacao,valor_pago,procedimentos_urgencia,fraude_tipo
0,9070,2109,652,208,8,-1,2,71821,0,217,997.29,0,regular
1,2604,5020,361,687,2,-1,1,44588,0,0,472.11,0,phantom_billing
2,7739,2232,675,209,7,0,2,43719,2,116,611.00,0,regular
3,1580,4568,231,201,7,-1,0,81804,2,506,799.38,1,regular
4,5059,1913,440,170,1,1,1,86624,1,657,832.95,0,regular
5,5754,5951,526,432,8,-1,1,47888,0,279,391.90,0,regular
6,3549,9328,797,246,3,0,1,62105,0,60,818.43,0,regular
7,410,3427,997,155,1,-1,1,75391,0,190,460.53,0,regular
8,4098,8299,985,590,4,0,2,76839,0,99,505.25,0,regular
9,6462,6401,382,149,7,-1,1,15571,0,527,368.76,0,regular
